# Importing libraries and datasets

In [2]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings

warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('../kaggle/input/playground-series-s4e12/train.csv', index_col='id')
test = pd.read_csv('../kaggle/input/playground-series-s4e12/test.csv', index_col='id')
sample = pd.read_csv('../kaggle/input/playground-series-s4e12/sample_submission.csv')
original = pd.read_csv('../kaggle/input//Insurance Premium Prediction Dataset.csv')

In [5]:
train.columns = train.columns.str.lower().str.replace(' ', '_')
test.columns = test.columns.str.lower().str.replace(' ', '_')
original.columns = original.columns.str.lower().str.replace(' ', '_')

In [8]:
original.policy_start_date.isin(train.policy_start_date).sum() / len(original)

0.9423330703578857

In [10]:
original = original[train.columns]

In [11]:
original

,age,gender,annual_income,marital_status,number_of_dependents,education_level,occupation,health_score,location,policy_type,previous_claims,vehicle_age,credit_score,insurance_duration,policy_start_date,customer_feedback,smoking_status,exercise_frequency,property_type,premium_amount
0,56.0,Male,99990.0,Married,1.0,Master's,NaN,31.074627,Urban,Comprehensive,NaN,13,320.0,5,2022-12-10 15:21:39.078837,Poor,Yes,Daily,Condo,308.0
1,46.0,Male,2867.0,Single,1.0,Bachelor's,NaN,50.271335,Urban,Comprehensive,NaN,3,694.0,4,2023-01-31 15:21:39.078837,Good,Yes,Monthly,House,517.0
2,32.0,Female,30154.0,Divorced,3.0,Bachelor's,NaN,14.714909,Suburban,Comprehensive,2.0,16,652.0,8,2023-11-26 15:21:39.078837,Poor,No,Monthly,House,849.0
3,60.0,Female,48371.0,Divorced,0.0,PhD,Self-Employed,25.346926,Rural,Comprehensive,1.0,11,330.0,7,2023-02-27 15:21:39.078837,Poor,No,Rarely,Condo,927.0
4,25.0,Female,54174.0,Divorced,0.0,High School,Self-Employed,6.659499,Urban,Comprehensive,NaN,9,NaN,8,2020-11-25 15:21:39.078837,Poor,No,Rarely,Condo,303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278855,53.0,Male,22594.0,Divorced,4.0,Master's,NaN,24.455533,Suburban,Premium,2.0,2,500.0,8,2022-04-26 15:21:39.291118,NaN,Yes,Daily,Apartment,367.0
278856,19.0,Female,NaN,Divorced,4.0,Master's,Unemployed,25.239104,Suburban,Basic,NaN,11,558.0,3,2022-11-13 15:21:39.291118,NaN,Yes,Monthly,Condo,3044.0
278857,45.0,Female,557.0,Single,1.0,Bachelor's,Self-Employed,62.112230,Rural,Premium,NaN,19,751.0,7,2021-01-20 15:21:39.291118,Poor,Yes,Weekly,Apartment,1334.0
278858,60.0,Male,4064.0,Single,4.0,Master's,Employed,6.163919,Rural,Premium,NaN,17,457.0,3,2024-01-14 15:21:39.291118,Average,Yes,Weekly,Apartment,662.0


In [4]:
train.head(1) # target - premium_amount

,age,gender,annual_income,marital_status,number_of_dependents,education_level,occupation,health_score,location,policy_type,previous_claims,vehicle_age,credit_score,insurance_duration,policy_start_date,customer_feedback,smoking_status,exercise_frequency,property_type,premium_amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0


In [5]:
train.describe().round(2)

,age,annual_income,number_of_dependents,health_score,previous_claims,vehicle_age,credit_score,insurance_duration,premium_amount
count,1181295.00,1155051.00,1090328.00,1125924.00,835971.00,1199994.00,1062118.00,1199999.00,1200000.00
mean,41.15,32745.22,2.01,25.61,1.00,9.57,592.92,5.02,1102.54
std,13.54,32179.51,1.42,12.20,0.98,5.78,149.98,2.59,865.00
min,18.00,1.00,0.00,2.01,0.00,0.00,300.00,1.00,20.00
25%,30.00,8001.00,1.00,15.92,0.00,5.00,468.00,3.00,514.00
50%,41.00,23911.00,2.00,24.58,1.00,10.00,595.00,5.00,872.00
75%,53.00,44634.00,3.00,34.53,2.00,15.00,721.00,7.00,1509.00
max,64.00,149997.00,4.00,58.98,9.00,19.00,849.00,9.00,4999.00


# Functions to work with features

In [12]:
from datetime import timedelta

def feature_transformer(df):

    df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])

    df['year'] = df['policy_start_date'].dt.year
    df['day'] = df['policy_start_date'].dt.day
    df['month'] = df['policy_start_date'].dt.month
    df['day_of_year'] = df['policy_start_date'].dt.dayofyear
    df['day_of_week'] = df['policy_start_date'].dt.weekday
    df['sin_day_of_week'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['cos_day_of_week'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df["seconds since 1970"] = df['policy_start_date'].astype("int64") // 10**9
    
    df['days_passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days
    

    policy_starts_min = df['policy_start_date'].min()  # 2019-08-17
    year = policy_starts_min.year  
    
    if policy_starts_min >= pd.Timestamp(f'{year}-01-01'):
        fiscal_year_start = pd.Timestamp(f'{year}-01-01')
    else:
        fiscal_year_start = pd.Timestamp(f'{year-1}-01-01')

    
    df['time_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.days
    df['seconds_from_fiscal_year'] = (df['policy_start_date'] - fiscal_year_start).dt.total_seconds()

    new_date = policy_starts_min - timedelta(days=1)
    df['time_from_first_policy'] = (df['policy_start_date'] - new_date).dt.days
    
    df['time_from_first_policy_seconds'] = (df['policy_start_date'] - new_date).dt.total_seconds()
    
    df['Days Passed'] = (df['policy_start_date'].max() - df['policy_start_date']).dt.days

    df['claims_vs_duration'] = df['previous_claims'] / df['insurance_duration']
    df['days_from_2019_crisis'] = (df['policy_start_date'] - pd.Timestamp('2019-01-01')).dt.days
    df['revenue_per_dependent'] = (df['annual_income'] / df['number_of_dependents'] + 1)
    df['ratio_of_doubts'] = (df['previous_claims'] + 1) / df['annual_income'] # NEW
    

    df.drop(columns=['time_from_first_policy', 'policy_start_date'], inplace=True)

    return df

In [13]:
def columns_imputer(df):
    for c in df.columns:
        df[f"is_{c}_na"] = df[c].isna().astype(int)

    return df

In [14]:
num_cols = ['age', 'annual_income', 'number_of_dependents', 'health_score','previous_claims', 'vehicle_age', 'credit_score', 'insurance_duration']

for col in num_cols:
    print(f"Column {col} has {train[train[col] == 0].shape[0]} zeros")

Column age has 0 zeros
Column annual_income has 0 zeros
Column number_of_dependents has 218124 zeros
Column health_score has 0 zeros
Column previous_claims has 305433 zeros
Column vehicle_age has 61615 zeros
Column credit_score has 0 zeros
Column insurance_duration has 0 zeros


In [15]:
def num_to_cat(df):
    global num_cols

    for col in num_cols:
        df[f"cat_{col}"] = df[col].astype('string')

    return df

In [16]:
# for col in num_cols:
#     sns.displot(train, x=col, kind="kde")  
#     plt.title(f"Distribution of {col}")
#     plt.show()

# Model preparation

In [17]:
X = train.drop('premium_amount', axis=1)
y = train['premium_amount']

y_log1p = np.log1p(y)


cats = X.select_dtypes(exclude=['number']).columns.tolist()
vars = X.select_dtypes(include=['number']).columns.tolist()
X = columns_imputer(X)

X[cats] = X[cats].fillna('None').astype('string')
X[vars] = X[vars].fillna(-999).astype(float)

X = feature_transformer(X)
X = num_to_cat(X)
#X = skewed_to_normal(X)

In [19]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from catboost import CatBoostRegressor, Pool

class CatboostWithOptuna:
    def __init__(self, X, y, n_folds=10, random_state=42, verbose=0, task_type='GPU', optuna_params=None):
        self.X = X
        self.y = y
        self.n_folds = n_folds
        self.random_state = random_state
        self.verbose = verbose
        self.task_type = task_type
        self.best_model_params = {}

        # Placeholders
        self.oof_predictions = []  # Out-of-fold predictions
        self.catboost_params_history = []  # История гиперпараметров
        self.best_score = float('inf')  # Лучший результат

    def _suggest_hyperparams(self, trial: optuna.Trial):
        return {
            'iterations' : trial.suggest_int('iterations', 400, 1000),
            'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.4, log=True),
            'depth' : trial.suggest_int('depth', 4, 9),
            'l2_leaf_reg' : trial.suggest_float('l2_leaf_reg', 0.1, 1, log=True),
            'loss_function' : trial.suggest_categorical('loss_function', ['RMSE']),
            'random_strength' : trial.suggest_float('random_strength', 1e-3, 1, log=True),
            'bagging_temperature' : trial.suggest_float('bagging_temperature', 1e-2, 1, log=True)
        }
    
    def objective(self, trial):
        
        params = self._suggest_hyperparams(trial)

        cat_features = self.X.select_dtypes(exclude='number').columns.tolist()

        # K-fold CV
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)
        fold_rmsle = []
        oof_preds = np.zeros((self.X.shape[0]))

        for fold, (train_index, val_index) in enumerate(kf.split(self.X)):
            print(f"Fold {fold + 1} out of {self.n_folds}")

            X_train, X_val = self.X.iloc[train_index], self.X.iloc[val_index]
            y_train, y_val = self.y.iloc[train_index], self.y.iloc[val_index]

            # Model creation 
            model = CatBoostRegressor(
                iterations=params['iterations'],
                learning_rate=params['learning_rate'],
                depth=params['depth'],
                l2_leaf_reg=params['l2_leaf_reg'],
                loss_function=params['loss_function'],
                random_strength=params['random_strength'],
                bagging_temperature=params['bagging_temperature'],
                verbose=0,
                gpu_ram_part=0.95,
                task_type=self.task_type
            )
            
            model.fit(X_train, y_train, cat_features=cat_features)

            val_pred = model.predict(X_val)

            oof_preds[val_index] = val_pred

            fold_rmsle.append(mean_squared_error(y_val, val_pred, squared=False))
        
        # save oof predictions
        self.oof_predictions.append(oof_preds.copy())

        mean_rmsle = np.mean(fold_rmsle)
        self.catboost_params_history.append({**params, 'RMSE': mean_rmsle})

        return mean_rmsle

        


    def optimize(self,n_trials=10):
        study = optuna.create_study(direction='minimize')
        study.optimize(self.objective, n_trials=n_trials)
        self.best_model_params = study.best_params
        self.best_score = study.best_value

        return study

    @property
    def oof(self):
        if not self.oof_predictions:
            raise ValueError("No OOF preds available")
        return self.oof_predictions

# OOF PREDICTIONS AND HYPERPARAMETERS SEARCH

In [13]:
y_log1p = np.log1p(y)

hyperparam_search = CatboostWithOptuna(X, y_log1p, n_folds = 5)
first_study = hyperparam_search.optimize(n_trials=40) # n_trials=50 is good

[I 2024-12-30 22:49:37,586] A new study created in memory with name: no-name-2bbdae00-eaa5-43b6-bc2b-27898cd70f61


Fold 1 out of 5
Fold 2 out of 5


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

y_train_log1p = np.log1p(y_train)
y_test_log1p = np.log1p(y_test)

In [15]:
# train_pool = Pool(X_train, y_train_log1p, feature_names = X_train.columns.tolist(), cat_features=cat_features)
# test_pool = Pool(X_test, y_test_log1p, feature_names = X_test.columns.tolist(), cat_features=cat_features)


# model = CatBoostRegressor(**first_study.best_params,
#         random_state=42,
#         task_type='GPU',
#         verbose = 0,
#         cat_features=cat_features)

# model.fit(X_train, y_train)

In [16]:
# feature_importances = model.get_feature_importance(train_pool, type='FeatureImportance')
# feature_names = X_train.columns


# plt.figure(figsize=(20, 10))
# sns.barplot(x=feature_importances, y=feature_names)
# plt.title("Feature Importances")
# plt.show()

In [17]:
oof_preds = hyperparam_search.oof_predictions

# Sum of predictions from all trials
mean_squared_log_error(y, sum(np.expm1(oof_preds))/len(oof_preds), squared = False)

1.0316888415327061

In [18]:
#Function for creating oof predictions (on test data)
def cv_test_preds(X, y, X_test,n_folds=5):

    folds = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    test_preds = []

    for fold, (train_idx, val_idx) in enumerate(folds.split(X, y)):

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]

        # Finding categorical features
        cat_features = X_test.select_dtypes(exclude=['number']).columns.tolist()


        model = CatBoostRegressor(**first_study.best_params,
        random_state=42,
        task_type='GPU',
        verbose = 0,
        cat_features=cat_features)
        model.fit(X_train, y_train, cat_features=cat_features)

        # Making predictions
        test_preds.append(model.predict(X_test))

    return test_preds

In [19]:
X_test = test
X_test = columns_imputer(X_test)

cats = X_test.select_dtypes(exclude=['number']).columns.tolist()
vars = X_test.select_dtypes(include=['number']).columns.tolist()

X_test[cats] = X_test[cats].fillna('None').astype('string')
X_test[vars] = X_test[vars].fillna(-999).astype(float)

X_test = feature_transformer(X_test)
X_test = num_to_cat(X_test)

In [20]:
X_final_preds = cv_test_preds(X,y_log1p,X_test, n_folds = 5)

preds = sum(np.expm1(X_final_preds))/len(X_final_preds)

In [21]:
def cv_test_preds(X, y, X_test, n_folds=5):
    folds = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    test_preds = []

    for fold, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]

        cat_features = X.select_dtypes(exclude=['number']).columns.tolist()

        model = CatBoostRegressor(
            **first_study.best_params,
            random_state=42,
            task_type='GPU',
            verbose=0,
            cat_features=cat_features
        )
        model.fit(X_train, y_train, cat_features=cat_features)

        test_preds.append(model.predict(X_test))

    return test_preds

In [22]:
X_final_preds = cv_test_preds(X, y_log1p, X_test, n_folds=5)

preds = sum(np.expm1(X_final_preds))/len(X_final_preds)

In [23]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

sample_submission['Premium Amount'] = preds
sample_submission.to_csv("submission.csv", index = False)

print('DONE!')

DONE!
